# Prepare raw dataset for Capstone project

Notebook used to create the JSON datasource of the project from `football_data_transfer_market`

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, when
import etl

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read CSV") \
    .master("local[*]") \
    .getOrCreate()

23/04/13 01:25:25 WARN Utils: Your hostname, exflqr43268 resolves to a loopback address: 127.0.1.1; using 192.168.0.215 instead (on interface wlp59s0)
23/04/13 01:25:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 01:25:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/13 01:25:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Convert SQLITE to JSON dataset

## Test number of combined rows for both dataset sources.

In [ ]:
from pathlib import Path

directory_path = Path("football_data_transfer_market")

csv_files = directory_path.glob("*.csv")
total_count = 0
count = 0
for csv in csv_files:
    if "player_valuations" in str(csv) or "events" in str(csv):
        continue
    # Read the CSV file into a DataFrame
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(str(csv))
    df_count = df.count()
    count += df_count
    print(csv.stem, df_count)
print("`football_data_transfer_market` rows count:", count)
total_count += count

competitions 43
games 61197
clubs 401
players 27456


appearances 1049585
club_games 122394
`football_data_transfer_market` rows count: 1261076


In [5]:
# This can be added into a json separate dataset.
players_df = etl.read_csv("football_data_transfer_market/players.csv", spark)
player_appearances = etl.read_csv("football_data_transfer_market/appearances.csv", spark)
game_events = etl.read_csv("football_data_transfer_market/game_events.csv", spark)

df = players_df.join(player_appearances, "player_id", "left").join(game_events, ["player_id", "game_id"], "left")

In [14]:
# Save the data in JSON format
df.write.format("json").save("raw_data/players_appearances")